In [29]:
from torchvision.models import resnet50
import torch
import torchvision
from torch import nn

In [30]:
def get_resnet():
    m = resnet50(weights=None)
    m.fc = nn.Linear(2048, 65)
    return m
model = get_resnet()

In [31]:
#from google.colab import drive
#drive.mount('/content/drive')
#!unzip '/content/drive/MyDrive/OfficeHomeDataset_10072016.zip' -d '/content'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class OfficeHomeDataset(torchvision.datasets.ImageFolder):

    def __init__(self, root_dir, domain, transform=None):
        super(OfficeHomeDataset, self).__init__(root=root_dir+"/"+domain, transform=transform)
        self.domain = domain

    def get_domain(self):
        return self.domain
    
      def __getitem__(self, idx):
        image_path = self.samples[idx][0]
        if isinstance(image_path, str):
            image = Image.open(image_path)
        else:
            image = image_path

        if self.transform:
            image = self.transform(image)

        label = torch.tensor(self.samples[idx][1])
        return image, label

In [ ]:
from torchvision import transforms
from PIL import Image
data_dir = "/content/drive/MyDrive/OfficeHome"
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

art_dataset = OfficeHomeDataset(data_dir, "Art", transform)

# Iterate through the dataset loader
train_loader = torch.utils.data.DataLoader(art_dataset, batch_size=32, shuffle=True)

In [ ]:
import torch.optim as optim

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
def update():
    for images, labels in train_loader:
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        return loss.item()

for epoch in range(10):
    loss_ = update()
    print(f"Epoch {epoch + 1}: Training loss = {loss_:.4f}")